In [1]:
import networkx as nx
import numpy as np 

In [2]:
remove_fraction = 0.1
attribute = "major_index"
max_iter = 100
eps = 1e-3
alpha = 0.5
data = "data/MIT8.gml"
iterations = 10

for attribute in ["major_index", "dorm", "year", "gender"]:
    for remove_fraction in [0.1, 0.2, 0.3, 0.4]:
        score = []
        for _ in range(iterations):
            graph = nx.read_gml(data)
            graph_original = nx.read_gml(data)

            nodes = list(graph.nodes())
            np.random.shuffle(nodes)
            removed = []
            for node in nodes[:int(len(nodes) * remove_fraction)]:
                if attribute in graph.nodes[node]:
                    removed.append(node)
                    graph.nodes[node][attribute] = 0

            a = nx.adjacency_matrix(graph)
            d_inv_sqrt = np.diag([1/np.sqrt(x) if x != 0 else 0 for x in np.sum(a, axis=1)])
            s = d_inv_sqrt @ a @ d_inv_sqrt

            # y_init = np.array([[1 if graph.nodes[node][attribute] == 1 else 0, 1 if graph.nodes[node][attribute] == 2 else 0] for node in graph])
            from sklearn.preprocessing import OneHotEncoder
            encoder = OneHotEncoder()

            y_true = [[graph_original.nodes[node][attribute]] for node in graph]
            y_true = encoder.fit_transform(y_true).toarray()
            y_init = [[graph.nodes[node][attribute]] for node in graph]
            y_init = encoder.transform(y_init).toarray()
            y_init = y_init[:,1:]
            y_true = y_true[:,1:]

            y = y_init.copy()

            for _ in range(max_iter):
                y_new = alpha * s @ y + (1 - alpha) * y_init
                if np.linalg.norm(y - y_new, 1) < eps:  
                    break
                y = y_new

            # for idx, node in enumerate(graph.nodes()):
            #     graph.nodes[node][attribute] = np.argmax(y[idx]) + 1

            success, failures = 0, 0
            for node in graph:
                if node in removed:
                    if y_true[int(node)][np.argmax(y[int(node)])] == 1:
                        success += 1
                    else:
                        failures += 1

            score.append(success / (success + failures))
        print(f"Attribute: {attribute}, remove fraction: {remove_fraction}, score: {np.mean(score)}")

Attribute: major_index, remove fraction: 0.1, score: 0.2576086956521739
Attribute: major_index, remove fraction: 0.2, score: 0.2533385093167702
Attribute: major_index, remove fraction: 0.3, score: 0.2494824016563147
Attribute: major_index, remove fraction: 0.4, score: 0.2516692546583851
Attribute: dorm, remove fraction: 0.1, score: 0.5347826086956522
Attribute: dorm, remove fraction: 0.2, score: 0.5354813664596273
Attribute: dorm, remove fraction: 0.3, score: 0.5339026915113871
Attribute: dorm, remove fraction: 0.4, score: 0.5267857142857143


KeyboardInterrupt: 

## Caltech

Attribute: major_index, remove fraction: 0.1, score: 0.23157894736842105
Attribute: major_index, remove fraction: 0.2, score: 0.23137254901960783
Attribute: major_index, remove fraction: 0.3, score: 0.21000000000000002
Attribute: major_index, remove fraction: 0.4, score: 0.19153094462540718
Attribute: dorm, remove fraction: 0.1, score: 0.6486842105263158
Attribute: dorm, remove fraction: 0.2, score: 0.6535947712418302
Attribute: dorm, remove fraction: 0.3, score: 0.6630434782608695
Attribute: dorm, remove fraction: 0.4, score: 0.6758957654723127
Attribute: year, remove fraction: 0.1, score: 0.6210526315789474
Attribute: year, remove fraction: 0.2, score: 0.6071895424836601
Attribute: year, remove fraction: 0.3, score: 0.5847826086956521
Attribute: year, remove fraction: 0.4, score: 0.6022801302931595
Attribute: gender, remove fraction: 0.1, score: 0.5960526315789474
Attribute: gender, remove fraction: 0.2, score: 0.580392156862745
Attribute: gender, remove fraction: 0.3, score: 0.6043478260869565
Attribute: gender, remove fraction: 0.4, score: 0.6065146579804561